## Retrieve VEX history
Parse online docs and find earliest Houdini version where the function was introduced.

In [1]:
import random
import bs4
import requests
from pprint import pprint
from IPython.display import HTML, Markdown

In [2]:
online = {}

r = requests.get('http://www.sidefx.com/docs/houdini17.0/vex/functions/')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['17.0'] = {a.text for a in s.find_all('a', class_='vex')}

r = requests.get('http://www.sidefx.com/docs/houdini16.5/vex/functions/')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['16.5'] = {a.text for a in s.find_all('a', class_='vex')}

r = requests.get('http://www.sidefx.com/docs/houdini16.0/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['16.0'] = {a.text for a in s.find_all('a', class_='vex')}

r = requests.get('http://www.sidefx.com/docs/houdini15.5/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['15.5'] = {a.text for a in s.find_all('a', class_='vex')}

r = requests.get('http://www.sidefx.com/docs/houdini15.0/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['15.0'] = {a.text for a in s.find(class_='hitlist').find_all('a')}

r = requests.get('http://www.sidefx.com/docs/houdini14.0/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['14.0'] = {td.find('a').text for td in s.find_all('td', class_='linktitlecell')}

r = requests.get('http://www.sidefx.com/docs/houdini13.0/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['13.0'] = {td.find('a').text for td in s.find_all('td', class_='linktitlecell')}

r = requests.get('http://www.sidefx.com/docs/houdini12.5/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['12.5'] = {td.find('a').text for td in s.find_all('td', class_='linktitlecell')}

r = requests.get('http://www.sidefx.com/docs/houdini12.1/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['12.1'] = {td.find('a').text for td in s.find_all('td', class_='linktitlecell')}

r = requests.get('http://www.sidefx.com/docs/houdini12.0/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['12.0'] = {td.find('a').text for td in s.find_all('td', class_='linktitlecell')}

r = requests.get('http://www.sidefx.com/docs/houdini11.0/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['11.0'] = {td.find('a').text for td in s.find_all('td', class_='linktitlecell')}

r = requests.get('http://www.sidefx.com/docs/houdini10.0/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['10.0'] = {a.text for a in s.find_all('a', class_='iconlink')}

# Rebuild dictionary so only unique function with earliest possible version left.
history = {
    '17.0' : sorted(online['17.0'] - online['16.5']),
    '16.5' : sorted((online['16.5'] - online['16.0']) & online['17.0']),
    '16.0' : sorted((online['16.0'] - online['15.5']) & online['17.0']),
    '15.5' : sorted((online['15.5'] - online['15.0']) & online['17.0']),
    '15.0' : sorted((online['15.0'] - online['14.0']) & online['17.0']),
    '14.0' : sorted((online['14.0'] - online['13.0']) & online['17.0']),
    '13.0' : sorted((online['13.0'] - online['12.5']) & online['17.0']),
    '12.5' : sorted((online['12.5'] - online['12.1']) & online['17.0']),
    '12.1' : sorted((online['12.1'] - online['12.0']) & online['17.0']),
    '12.0' : sorted((online['12.0'] - online['11.0']) & online['17.0']),
    '11.0' : sorted((online['11.0'] - online['10.0']) & online['17.0'])
}
history = {f: {'added': v} for v, funs in history.items() for f in funs}

def test():
    f = random.choice(list(history.keys()))
    print(
        'Total history records: %d' % len(history),
        '\nRandom record: %s' % f,
    )
    pprint(history[f])

test()

Total history records: 742 
Random record: primattribtypeinfo
{'added': '14.0'}


In [3]:
# Make some history information to be visible in online Notebook viewers.
def diff(a, b):
    return sorted(online[b] - online[a])

def link(version, function):
    return ('[{function}](http://www.sidefx.com/docs/houdini{version}/vex/functions/{function})'.format(**locals()))

def block(new, old):
    removed = '\n'.join('-' + link(old, f) for f in diff(new, old))
    added = '\n'.join('+' + link(new, f) for f in diff(old, new))
    total = len(online[new])
    return '''
#### New in {new}

{removed}{added}

Total: {total} functions.
    '''.format(**locals())

markup = '### VEX History'
markup += block('17.0', '16.5')
markup += block('16.5', '16.0')
markup += block('16.0', '15.5')
markup += block('15.5', '15.0')
markup += block('15.0', '14.0')
markup += block('14.0', '13.0')
markup += block('13.0', '12.5')
markup += block('12.5', '12.1')
markup += block('12.1', '12.0')
markup += block('12.0', '11.0')
markup += block('11.0', '10.0')
markup += '''
#### From 10.0

{}

Total: {} functions.'''.format('\n'.join([link('10.0', f) for f in sorted(online['10.0'] & online['17.0'])]),
                               len(online['10.0'] & online['17.0']))

Markdown(markup)

### VEX History
#### New in 17.0

+[agentsolvefbik](http://www.sidefx.com/docs/houdini17.0/vex/functions/agentsolvefbik)
+[attribdataid](http://www.sidefx.com/docs/houdini17.0/vex/functions/attribdataid)
+[chop](http://www.sidefx.com/docs/houdini17.0/vex/functions/chop)
+[chopt](http://www.sidefx.com/docs/houdini17.0/vex/functions/chopt)
+[diagonalizesymmetric](http://www.sidefx.com/docs/houdini17.0/vex/functions/diagonalizesymmetric)
+[expandvertexgroup](http://www.sidefx.com/docs/houdini17.0/vex/functions/expandvertexgroup)
+[getpackedtransform](http://www.sidefx.com/docs/houdini17.0/vex/functions/getpackedtransform)
+[islpeactive](http://www.sidefx.com/docs/houdini17.0/vex/functions/islpeactive)
+[packedtransform](http://www.sidefx.com/docs/houdini17.0/vex/functions/packedtransform)
+[pxnoised](http://www.sidefx.com/docs/houdini17.0/vex/functions/pxnoised)
+[random_poisson](http://www.sidefx.com/docs/houdini17.0/vex/functions/random_poisson)
+[sample_circle_ring_uniform](http://www.sidefx.com/docs/houdini17.0/vex/functions/sample_circle_ring_uniform)
+[sample_sphere_shell_uniform](http://www.sidefx.com/docs/houdini17.0/vex/functions/sample_sphere_shell_uniform)
+[setpackedtransform](http://www.sidefx.com/docs/houdini17.0/vex/functions/setpackedtransform)
+[solvefbik](http://www.sidefx.com/docs/houdini17.0/vex/functions/solvefbik)
+[surfacedist](http://www.sidefx.com/docs/houdini17.0/vex/functions/surfacedist)
+[uniquevals](http://www.sidefx.com/docs/houdini17.0/vex/functions/uniquevals)

Total: 742 functions.
    
#### New in 16.5

-[arraylength](http://www.sidefx.com/docs/houdini16.0/vex/functions/arraylength)
-[emission_bsdf](http://www.sidefx.com/docs/houdini16.0/vex/functions/emission_bsdf)
-[foreach](http://www.sidefx.com/docs/houdini16.0/vex/functions/foreach)
-[forpoints](http://www.sidefx.com/docs/houdini16.0/vex/functions/forpoints)
-[gather](http://www.sidefx.com/docs/houdini16.0/vex/functions/gather)
-[gradient3d](http://www.sidefx.com/docs/houdini16.0/vex/functions/gradient3d)
-[illuminance](http://www.sidefx.com/docs/houdini16.0/vex/functions/illuminance)
-[importance_light](http://www.sidefx.com/docs/houdini16.0/vex/functions/importance_light)
-[integrate3d](http://www.sidefx.com/docs/houdini16.0/vex/functions/integrate3d)
-[integrate3dClip](http://www.sidefx.com/docs/houdini16.0/vex/functions/integrate3dClip)
-[intersect3d](http://www.sidefx.com/docs/houdini16.0/vex/functions/intersect3d)
-[istracing](http://www.sidefx.com/docs/houdini16.0/vex/functions/istracing)+[agentfindtransformgroup](http://www.sidefx.com/docs/houdini16.5/vex/functions/agentfindtransformgroup)
+[agenttransformgroupweight](http://www.sidefx.com/docs/houdini16.5/vex/functions/agenttransformgroupweight)
+[ch2](http://www.sidefx.com/docs/houdini16.5/vex/functions/ch2)
+[chexpr](http://www.sidefx.com/docs/houdini16.5/vex/functions/chexpr)
+[chexprf](http://www.sidefx.com/docs/houdini16.5/vex/functions/chexprf)
+[chexprt](http://www.sidefx.com/docs/houdini16.5/vex/functions/chexprt)
+[chu](http://www.sidefx.com/docs/houdini16.5/vex/functions/chu)
+[cwnoise](http://www.sidefx.com/docs/houdini16.5/vex/functions/cwnoise)
+[mwnoise](http://www.sidefx.com/docs/houdini16.5/vex/functions/mwnoise)
+[ocio_activedisplays](http://www.sidefx.com/docs/houdini16.5/vex/functions/ocio_activedisplays)
+[ocio_activeviews](http://www.sidefx.com/docs/houdini16.5/vex/functions/ocio_activeviews)
+[ocio_roles](http://www.sidefx.com/docs/houdini16.5/vex/functions/ocio_roles)

Total: 725 functions.
    
#### New in 16.0

-[getuvobjects](http://www.sidefx.com/docs/houdini15.5/vex/functions/getuvobjects)+[agentcliptransformgroups](http://www.sidefx.com/docs/houdini16.0/vex/functions/agentcliptransformgroups)
+[agenttransformgroupmember](http://www.sidefx.com/docs/houdini16.0/vex/functions/agenttransformgroupmember)
+[agenttransformgroups](http://www.sidefx.com/docs/houdini16.0/vex/functions/agenttransformgroups)
+[chadd](http://www.sidefx.com/docs/houdini16.0/vex/functions/chadd)
+[chattr](http://www.sidefx.com/docs/houdini16.0/vex/functions/chattr)
+[chattrnames](http://www.sidefx.com/docs/houdini16.0/vex/functions/chattrnames)
+[chindex](http://www.sidefx.com/docs/houdini16.0/vex/functions/chindex)
+[chinputlimits](http://www.sidefx.com/docs/houdini16.0/vex/functions/chinputlimits)
+[chnames](http://www.sidefx.com/docs/houdini16.0/vex/functions/chnames)
+[chreadbuf](http://www.sidefx.com/docs/houdini16.0/vex/functions/chreadbuf)
+[chremove](http://www.sidefx.com/docs/houdini16.0/vex/functions/chremove)
+[chremoveattr](http://www.sidefx.com/docs/houdini16.0/vex/functions/chremoveattr)
+[chrename](http://www.sidefx.com/docs/houdini16.0/vex/functions/chrename)
+[chresizebuf](http://www.sidefx.com/docs/houdini16.0/vex/functions/chresizebuf)
+[chsetattr](http://www.sidefx.com/docs/houdini16.0/vex/functions/chsetattr)
+[chsetlength](http://www.sidefx.com/docs/houdini16.0/vex/functions/chsetlength)
+[chsetrate](http://www.sidefx.com/docs/houdini16.0/vex/functions/chsetrate)
+[chsetstart](http://www.sidefx.com/docs/houdini16.0/vex/functions/chsetstart)
+[chwritebuf](http://www.sidefx.com/docs/houdini16.0/vex/functions/chwritebuf)
+[create_cdf](http://www.sidefx.com/docs/houdini16.0/vex/functions/create_cdf)
+[create_pdf](http://www.sidefx.com/docs/houdini16.0/vex/functions/create_pdf)
+[efit](http://www.sidefx.com/docs/houdini16.0/vex/functions/efit)
+[expandedgegroup](http://www.sidefx.com/docs/houdini16.0/vex/functions/expandedgegroup)
+[geounwrap](http://www.sidefx.com/docs/houdini16.0/vex/functions/geounwrap)
+[getbbox_center](http://www.sidefx.com/docs/houdini16.0/vex/functions/getbbox_center)
+[getbbox_max](http://www.sidefx.com/docs/houdini16.0/vex/functions/getbbox_max)
+[getbbox_min](http://www.sidefx.com/docs/houdini16.0/vex/functions/getbbox_min)
+[getbbox_size](http://www.sidefx.com/docs/houdini16.0/vex/functions/getbbox_size)
+[getgroupid](http://www.sidefx.com/docs/houdini16.0/vex/functions/getgroupid)
+[getlocalcurvature](http://www.sidefx.com/docs/houdini16.0/vex/functions/getlocalcurvature)
+[getmaterialid](http://www.sidefx.com/docs/houdini16.0/vex/functions/getmaterialid)
+[getobjectid](http://www.sidefx.com/docs/houdini16.0/vex/functions/getobjectid)
+[getpointbbox_center](http://www.sidefx.com/docs/houdini16.0/vex/functions/getpointbbox_center)
+[getpointbbox_max](http://www.sidefx.com/docs/houdini16.0/vex/functions/getpointbbox_max)
+[getpointbbox_min](http://www.sidefx.com/docs/houdini16.0/vex/functions/getpointbbox_min)
+[getpointbbox_size](http://www.sidefx.com/docs/houdini16.0/vex/functions/getpointbbox_size)
+[hasmetadata](http://www.sidefx.com/docs/houdini16.0/vex/functions/hasmetadata)
+[inedgegroup](http://www.sidefx.com/docs/houdini16.0/vex/functions/inedgegroup)
+[lightbounces](http://www.sidefx.com/docs/houdini16.0/vex/functions/lightbounces)
+[mask_bsdf](http://www.sidefx.com/docs/houdini16.0/vex/functions/mask_bsdf)
+[metadata](http://www.sidefx.com/docs/houdini16.0/vex/functions/metadata)
+[nedgesgroup](http://www.sidefx.com/docs/houdini16.0/vex/functions/nedgesgroup)
+[ninputs](http://www.sidefx.com/docs/houdini16.0/vex/functions/ninputs)
+[ocean_sample](http://www.sidefx.com/docs/houdini16.0/vex/functions/ocean_sample)
+[ocio_import](http://www.sidefx.com/docs/houdini16.0/vex/functions/ocio_import)
+[ocio_spaces](http://www.sidefx.com/docs/houdini16.0/vex/functions/ocio_spaces)
+[ocio_transform](http://www.sidefx.com/docs/houdini16.0/vex/functions/ocio_transform)
+[opparentbonetransform](http://www.sidefx.com/docs/houdini16.0/vex/functions/opparentbonetransform)
+[opparenttransform](http://www.sidefx.com/docs/houdini16.0/vex/functions/opparenttransform)
+[opparmtransform](http://www.sidefx.com/docs/houdini16.0/vex/functions/opparmtransform)
+[oppreconstrainttransform](http://www.sidefx.com/docs/houdini16.0/vex/functions/oppreconstrainttransform)
+[oppretransform](http://www.sidefx.com/docs/houdini16.0/vex/functions/oppretransform)
+[osd_lookupface](http://www.sidefx.com/docs/houdini16.0/vex/functions/osd_lookupface)
+[osd_lookuppatch](http://www.sidefx.com/docs/houdini16.0/vex/functions/osd_lookuppatch)
+[planepointdistance](http://www.sidefx.com/docs/houdini16.0/vex/functions/planepointdistance)
+[planesphereintersect](http://www.sidefx.com/docs/houdini16.0/vex/functions/planesphereintersect)
+[predicate_incircle](http://www.sidefx.com/docs/houdini16.0/vex/functions/predicate_incircle)
+[predicate_insphere](http://www.sidefx.com/docs/houdini16.0/vex/functions/predicate_insphere)
+[predicate_orient2d](http://www.sidefx.com/docs/houdini16.0/vex/functions/predicate_orient2d)
+[predicate_orient3d](http://www.sidefx.com/docs/houdini16.0/vex/functions/predicate_orient3d)
+[primarclen](http://www.sidefx.com/docs/houdini16.0/vex/functions/primarclen)
+[primduv](http://www.sidefx.com/docs/houdini16.0/vex/functions/primduv)
+[primfind](http://www.sidefx.com/docs/houdini16.0/vex/functions/primfind)
+[primuvconvert](http://www.sidefx.com/docs/houdini16.0/vex/functions/primuvconvert)
+[sample_cdf](http://www.sidefx.com/docs/houdini16.0/vex/functions/sample_cdf)
+[scatter](http://www.sidefx.com/docs/houdini16.0/vex/functions/scatter)
+[setagentclips](http://www.sidefx.com/docs/houdini16.0/vex/functions/setagentclips)
+[setedgegroup](http://www.sidefx.com/docs/houdini16.0/vex/functions/setedgegroup)
+[solid_angle](http://www.sidefx.com/docs/houdini16.0/vex/functions/solid_angle)
+[split_bsdf](http://www.sidefx.com/docs/houdini16.0/vex/functions/split_bsdf)
+[sssapprox](http://www.sidefx.com/docs/houdini16.0/vex/functions/sssapprox)
+[uvdist](http://www.sidefx.com/docs/houdini16.0/vex/functions/uvdist)
+[uvintersect](http://www.sidefx.com/docs/houdini16.0/vex/functions/uvintersect)
+[uvsample](http://www.sidefx.com/docs/houdini16.0/vex/functions/uvsample)
+[volumevoxeldiameter](http://www.sidefx.com/docs/houdini16.0/vex/functions/volumevoxeldiameter)

Total: 725 functions.
    
#### New in 15.5

+[agentclipchannel](http://www.sidefx.com/docs/houdini15.5/vex/functions/agentclipchannel)
+[agentclipsample](http://www.sidefx.com/docs/houdini15.5/vex/functions/agentclipsample)
+[agentclipsamplerate](http://www.sidefx.com/docs/houdini15.5/vex/functions/agentclipsamplerate)
+[getderiv](http://www.sidefx.com/docs/houdini15.5/vex/functions/getderiv)
+[getsamplestore](http://www.sidefx.com/docs/houdini15.5/vex/functions/getsamplestore)
+[getuvobjects](http://www.sidefx.com/docs/houdini15.5/vex/functions/getuvobjects)
+[getuvtangents](http://www.sidefx.com/docs/houdini15.5/vex/functions/getuvtangents)
+[setsamplestore](http://www.sidefx.com/docs/houdini15.5/vex/functions/setsamplestore)
+[slideframe](http://www.sidefx.com/docs/houdini15.5/vex/functions/slideframe)

Total: 651 functions.
    
#### New in 15.0

-[addattribute](http://www.sidefx.com/docs/houdini14.0/vex/functions/addattribute)
-[addgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/addgroup)
-[bumpmap](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmap)
-[bumpmapA](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmapA)
-[bumpmapB](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmapB)
-[bumpmapG](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmapG)
-[bumpmapL](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmapL)
-[bumpmapR](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmapR)
-[file_eof](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_eof)
-[file_flush](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_flush)
-[file_open](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_open)
-[file_read](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_read)
-[file_readline](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_readline)
-[file_readlines](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_readlines)
-[file_seek](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_seek)
-[file_size](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_size)
-[file_tell](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_tell)
-[file_write](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_write)
-[file_writeline](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_writeline)
-[file_writelines](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_writelines)
-[import](http://www.sidefx.com/docs/houdini14.0/vex/functions/import)
-[ingroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/ingroup)
-[newgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/newgroup)
-[rawbumpmap](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmap)
-[rawbumpmapA](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmapA)
-[rawbumpmapB](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmapB)
-[rawbumpmapG](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmapG)
-[rawbumpmapL](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmapL)
-[rawbumpmapR](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmapR)
-[removegroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/removegroup)
-[variadicarguments](http://www.sidefx.com/docs/houdini14.0/vex/functions/variadicarguments)+[addvisualizer](http://www.sidefx.com/docs/houdini15.0/vex/functions/addvisualizer)
+[attrib](http://www.sidefx.com/docs/houdini15.0/vex/functions/attrib)
+[attribclass](http://www.sidefx.com/docs/houdini15.0/vex/functions/attribclass)
+[error](http://www.sidefx.com/docs/houdini15.0/vex/functions/error)
+[fuzzify](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzify)
+[fuzzy_and](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_and)
+[fuzzy_defuzz_centroid](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_defuzz_centroid)
+[fuzzy_nand](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_nand)
+[fuzzy_nor](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_nor)
+[fuzzy_not](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_not)
+[fuzzy_nxor](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_nxor)
+[fuzzy_or](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_or)
+[fuzzy_xor](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_xor)
+[getattrib](http://www.sidefx.com/docs/houdini15.0/vex/functions/getattrib)
+[getsmoothP](http://www.sidefx.com/docs/houdini15.0/vex/functions/getsmoothP)
+[hasattrib](http://www.sidefx.com/docs/houdini15.0/vex/functions/hasattrib)
+[idtopoint](http://www.sidefx.com/docs/houdini15.0/vex/functions/idtopoint)
+[idtoprim](http://www.sidefx.com/docs/houdini15.0/vex/functions/idtoprim)
+[invertexgroup](http://www.sidefx.com/docs/houdini15.0/vex/functions/invertexgroup)
+[nametopoint](http://www.sidefx.com/docs/houdini15.0/vex/functions/nametopoint)
+[nametoprim](http://www.sidefx.com/docs/houdini15.0/vex/functions/nametoprim)
+[nvertices](http://www.sidefx.com/docs/houdini15.0/vex/functions/nvertices)
+[nverticesgroup](http://www.sidefx.com/docs/houdini15.0/vex/functions/nverticesgroup)
+[opdigits](http://www.sidefx.com/docs/houdini15.0/vex/functions/opdigits)
+[opfullpath](http://www.sidefx.com/docs/houdini15.0/vex/functions/opfullpath)
+[pgfind](http://www.sidefx.com/docs/houdini15.0/vex/functions/pgfind)
+[product](http://www.sidefx.com/docs/houdini15.0/vex/functions/product)
+[relativepath](http://www.sidefx.com/docs/houdini15.0/vex/functions/relativepath)
+[resample_linear](http://www.sidefx.com/docs/houdini15.0/vex/functions/resample_linear)
+[setvertexgroup](http://www.sidefx.com/docs/houdini15.0/vex/functions/setvertexgroup)
+[setvertexpoint](http://www.sidefx.com/docs/houdini15.0/vex/functions/setvertexpoint)
+[smoothrotation](http://www.sidefx.com/docs/houdini15.0/vex/functions/smoothrotation)
+[splitpath](http://www.sidefx.com/docs/houdini15.0/vex/functions/splitpath)
+[sum](http://www.sidefx.com/docs/houdini15.0/vex/functions/sum)
+[uvunwrap](http://www.sidefx.com/docs/houdini15.0/vex/functions/uvunwrap)
+[warning](http://www.sidefx.com/docs/houdini15.0/vex/functions/warning)

Total: 642 functions.
    
#### New in 14.0

+[Du](http://www.sidefx.com/docs/houdini14.0/vex/functions/Du)
+[Dv](http://www.sidefx.com/docs/houdini14.0/vex/functions/Dv)
+[Dw](http://www.sidefx.com/docs/houdini14.0/vex/functions/Dw)
+[abs](http://www.sidefx.com/docs/houdini14.0/vex/functions/abs)
+[accessframe](http://www.sidefx.com/docs/houdini14.0/vex/functions/accessframe)
+[acos](http://www.sidefx.com/docs/houdini14.0/vex/functions/acos)
+[addattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/addattrib)
+[addattribute](http://www.sidefx.com/docs/houdini14.0/vex/functions/addattribute)
+[adddetailattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/adddetailattrib)
+[addgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/addgroup)
+[addpoint](http://www.sidefx.com/docs/houdini14.0/vex/functions/addpoint)
+[addpointattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/addpointattrib)
+[addprim](http://www.sidefx.com/docs/houdini14.0/vex/functions/addprim)
+[addprimattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/addprimattrib)
+[addvariablename](http://www.sidefx.com/docs/houdini14.0/vex/functions/addvariablename)
+[addvertex](http://www.sidefx.com/docs/houdini14.0/vex/functions/addvertex)
+[addvertexattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/addvertexattrib)
+[agentaddclip](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentaddclip)
+[agentclipcatalog](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentclipcatalog)
+[agentcliplength](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentcliplength)
+[agentclipnames](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentclipnames)
+[agentclipsamplelocal](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentclipsamplelocal)
+[agentclipsampleworld](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentclipsampleworld)
+[agentcliptimes](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentcliptimes)
+[agentclipweights](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentclipweights)
+[agentcollisionlayer](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentcollisionlayer)
+[agentcurrentlayer](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentcurrentlayer)
+[agentlayerbindings](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentlayerbindings)
+[agentlayers](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentlayers)
+[agentlayershapes](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentlayershapes)
+[agentlocaltransform](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentlocaltransform)
+[agentlocaltransforms](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentlocaltransforms)
+[agentrigchildren](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentrigchildren)
+[agentrigfind](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentrigfind)
+[agentrigparent](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentrigparent)
+[agenttransformcount](http://www.sidefx.com/docs/houdini14.0/vex/functions/agenttransformcount)
+[agenttransformnames](http://www.sidefx.com/docs/houdini14.0/vex/functions/agenttransformnames)
+[agenttransformtolocal](http://www.sidefx.com/docs/houdini14.0/vex/functions/agenttransformtolocal)
+[agenttransformtoworld](http://www.sidefx.com/docs/houdini14.0/vex/functions/agenttransformtoworld)
+[agentworldtransform](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentworldtransform)
+[agentworldtransforms](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentworldtransforms)
+[albedo](http://www.sidefx.com/docs/houdini14.0/vex/functions/albedo)
+[alphaname](http://www.sidefx.com/docs/houdini14.0/vex/functions/alphaname)
+[ambient](http://www.sidefx.com/docs/houdini14.0/vex/functions/ambient)
+[anoise](http://www.sidefx.com/docs/houdini14.0/vex/functions/anoise)
+[append](http://www.sidefx.com/docs/houdini14.0/vex/functions/append)
+[area](http://www.sidefx.com/docs/houdini14.0/vex/functions/area)
+[argsort](http://www.sidefx.com/docs/houdini14.0/vex/functions/argsort)
+[array](http://www.sidefx.com/docs/houdini14.0/vex/functions/array)
+[arraylength](http://www.sidefx.com/docs/houdini14.0/vex/functions/arraylength)
+[ashikhmin](http://www.sidefx.com/docs/houdini14.0/vex/functions/ashikhmin)
+[asin](http://www.sidefx.com/docs/houdini14.0/vex/functions/asin)
+[assert_enabled](http://www.sidefx.com/docs/houdini14.0/vex/functions/assert_enabled)
+[assign](http://www.sidefx.com/docs/houdini14.0/vex/functions/assign)
+[atan](http://www.sidefx.com/docs/houdini14.0/vex/functions/atan)
+[atan2](http://www.sidefx.com/docs/houdini14.0/vex/functions/atan2)
+[atof](http://www.sidefx.com/docs/houdini14.0/vex/functions/atof)
+[atoi](http://www.sidefx.com/docs/houdini14.0/vex/functions/atoi)
+[atten](http://www.sidefx.com/docs/houdini14.0/vex/functions/atten)
+[attribsize](http://www.sidefx.com/docs/houdini14.0/vex/functions/attribsize)
+[attribtype](http://www.sidefx.com/docs/houdini14.0/vex/functions/attribtype)
+[attribtypeinfo](http://www.sidefx.com/docs/houdini14.0/vex/functions/attribtypeinfo)
+[avg](http://www.sidefx.com/docs/houdini14.0/vex/functions/avg)
+[binput](http://www.sidefx.com/docs/houdini14.0/vex/functions/binput)
+[blackbody](http://www.sidefx.com/docs/houdini14.0/vex/functions/blackbody)
+[blinn](http://www.sidefx.com/docs/houdini14.0/vex/functions/blinn)
+[blinnBRDF](http://www.sidefx.com/docs/houdini14.0/vex/functions/blinnBRDF)
+[bouncelabel](http://www.sidefx.com/docs/houdini14.0/vex/functions/bouncelabel)
+[bouncemask](http://www.sidefx.com/docs/houdini14.0/vex/functions/bouncemask)
+[bumpmap](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmap)
+[bumpmapA](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmapA)
+[bumpmapB](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmapB)
+[bumpmapG](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmapG)
+[bumpmapL](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmapL)
+[bumpmapR](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmapR)
+[bumpname](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpname)
+[cbrt](http://www.sidefx.com/docs/houdini14.0/vex/functions/cbrt)
+[ceil](http://www.sidefx.com/docs/houdini14.0/vex/functions/ceil)
+[ch](http://www.sidefx.com/docs/houdini14.0/vex/functions/ch)
+[ch3](http://www.sidefx.com/docs/houdini14.0/vex/functions/ch3)
+[ch4](http://www.sidefx.com/docs/houdini14.0/vex/functions/ch4)
+[chend](http://www.sidefx.com/docs/houdini14.0/vex/functions/chend)
+[chendf](http://www.sidefx.com/docs/houdini14.0/vex/functions/chendf)
+[chendt](http://www.sidefx.com/docs/houdini14.0/vex/functions/chendt)
+[chf](http://www.sidefx.com/docs/houdini14.0/vex/functions/chf)
+[chi](http://www.sidefx.com/docs/houdini14.0/vex/functions/chi)
+[chinput](http://www.sidefx.com/docs/houdini14.0/vex/functions/chinput)
+[chname](http://www.sidefx.com/docs/houdini14.0/vex/functions/chname)
+[chnumchan](http://www.sidefx.com/docs/houdini14.0/vex/functions/chnumchan)
+[chp](http://www.sidefx.com/docs/houdini14.0/vex/functions/chp)
+[chr](http://www.sidefx.com/docs/houdini14.0/vex/functions/chr)
+[chramp](http://www.sidefx.com/docs/houdini14.0/vex/functions/chramp)
+[chrate](http://www.sidefx.com/docs/houdini14.0/vex/functions/chrate)
+[chs](http://www.sidefx.com/docs/houdini14.0/vex/functions/chs)
+[chsraw](http://www.sidefx.com/docs/houdini14.0/vex/functions/chsraw)
+[chstart](http://www.sidefx.com/docs/houdini14.0/vex/functions/chstart)
+[chstartf](http://www.sidefx.com/docs/houdini14.0/vex/functions/chstartf)
+[chstartt](http://www.sidefx.com/docs/houdini14.0/vex/functions/chstartt)
+[chv](http://www.sidefx.com/docs/houdini14.0/vex/functions/chv)
+[cinput](http://www.sidefx.com/docs/houdini14.0/vex/functions/cinput)
+[ckspline](http://www.sidefx.com/docs/houdini14.0/vex/functions/ckspline)
+[clamp](http://www.sidefx.com/docs/houdini14.0/vex/functions/clamp)
+[clip](http://www.sidefx.com/docs/houdini14.0/vex/functions/clip)
+[colormap](http://www.sidefx.com/docs/houdini14.0/vex/functions/colormap)
+[colorname](http://www.sidefx.com/docs/houdini14.0/vex/functions/colorname)
+[computenormal](http://www.sidefx.com/docs/houdini14.0/vex/functions/computenormal)
+[concat](http://www.sidefx.com/docs/houdini14.0/vex/functions/concat)
+[cone](http://www.sidefx.com/docs/houdini14.0/vex/functions/cone)
+[cos](http://www.sidefx.com/docs/houdini14.0/vex/functions/cos)
+[cosh](http://www.sidefx.com/docs/houdini14.0/vex/functions/cosh)
+[cracktransform](http://www.sidefx.com/docs/houdini14.0/vex/functions/cracktransform)
+[cross](http://www.sidefx.com/docs/houdini14.0/vex/functions/cross)
+[cspline](http://www.sidefx.com/docs/houdini14.0/vex/functions/cspline)
+[ctransform](http://www.sidefx.com/docs/houdini14.0/vex/functions/ctransform)
+[curlnoise](http://www.sidefx.com/docs/houdini14.0/vex/functions/curlnoise)
+[curlnoise2d](http://www.sidefx.com/docs/houdini14.0/vex/functions/curlnoise2d)
+[curlxnoise](http://www.sidefx.com/docs/houdini14.0/vex/functions/curlxnoise)
+[curlxnoise2d](http://www.sidefx.com/docs/houdini14.0/vex/functions/curlxnoise2d)
+[cvex_bsdf](http://www.sidefx.com/docs/houdini14.0/vex/functions/cvex_bsdf)
+[degrees](http://www.sidefx.com/docs/houdini14.0/vex/functions/degrees)
+[depthmap](http://www.sidefx.com/docs/houdini14.0/vex/functions/depthmap)
+[depthname](http://www.sidefx.com/docs/houdini14.0/vex/functions/depthname)
+[detail](http://www.sidefx.com/docs/houdini14.0/vex/functions/detail)
+[detailattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/detailattrib)
+[detailattribsize](http://www.sidefx.com/docs/houdini14.0/vex/functions/detailattribsize)
+[detailattribtype](http://www.sidefx.com/docs/houdini14.0/vex/functions/detailattribtype)
+[detailattribtypeinfo](http://www.sidefx.com/docs/houdini14.0/vex/functions/detailattribtypeinfo)
+[detailintrinsic](http://www.sidefx.com/docs/houdini14.0/vex/functions/detailintrinsic)
+[determinant](http://www.sidefx.com/docs/houdini14.0/vex/functions/determinant)
+[diffuse](http://www.sidefx.com/docs/houdini14.0/vex/functions/diffuse)
+[diffuseBRDF](http://www.sidefx.com/docs/houdini14.0/vex/functions/diffuseBRDF)
+[dihedral](http://www.sidefx.com/docs/houdini14.0/vex/functions/dihedral)
+[dimport](http://www.sidefx.com/docs/houdini14.0/vex/functions/dimport)
+[distance](http://www.sidefx.com/docs/houdini14.0/vex/functions/distance)
+[distance2](http://www.sidefx.com/docs/houdini14.0/vex/functions/distance2)
+[dot](http://www.sidefx.com/docs/houdini14.0/vex/functions/dot)
+[dsmpixel](http://www.sidefx.com/docs/houdini14.0/vex/functions/dsmpixel)
+[eigenvalues](http://www.sidefx.com/docs/houdini14.0/vex/functions/eigenvalues)
+[emission_bsdf](http://www.sidefx.com/docs/houdini14.0/vex/functions/emission_bsdf)
+[endswith](http://www.sidefx.com/docs/houdini14.0/vex/functions/endswith)
+[environment](http://www.sidefx.com/docs/houdini14.0/vex/functions/environment)
+[erf](http://www.sidefx.com/docs/houdini14.0/vex/functions/erf)
+[erf_inv](http://www.sidefx.com/docs/houdini14.0/vex/functions/erf_inv)
+[erfc](http://www.sidefx.com/docs/houdini14.0/vex/functions/erfc)
+[eulertoquaternion](http://www.sidefx.com/docs/houdini14.0/vex/functions/eulertoquaternion)
+[eval_bsdf](http://www.sidefx.com/docs/houdini14.0/vex/functions/eval_bsdf)
+[exp](http://www.sidefx.com/docs/houdini14.0/vex/functions/exp)
+[expand_udim](http://www.sidefx.com/docs/houdini14.0/vex/functions/expand_udim)
+[expandpointgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/expandpointgroup)
+[expandprimgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/expandprimgroup)
+[fastshadow](http://www.sidefx.com/docs/houdini14.0/vex/functions/fastshadow)
+[filamentsample](http://www.sidefx.com/docs/houdini14.0/vex/functions/filamentsample)
+[file_eof](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_eof)
+[file_flush](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_flush)
+[file_open](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_open)
+[file_read](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_read)
+[file_readline](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_readline)
+[file_readlines](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_readlines)
+[file_seek](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_seek)
+[file_size](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_size)
+[file_stat](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_stat)
+[file_tell](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_tell)
+[file_write](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_write)
+[file_writeline](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_writeline)
+[file_writelines](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_writelines)
+[filtershadow](http://www.sidefx.com/docs/houdini14.0/vex/functions/filtershadow)
+[filterstep](http://www.sidefx.com/docs/houdini14.0/vex/functions/filterstep)
+[find](http://www.sidefx.com/docs/houdini14.0/vex/functions/find)
+[findattribval](http://www.sidefx.com/docs/houdini14.0/vex/functions/findattribval)
+[findattribvalcount](http://www.sidefx.com/docs/houdini14.0/vex/functions/findattribvalcount)
+[finput](http://www.sidefx.com/docs/houdini14.0/vex/functions/finput)
+[fit](http://www.sidefx.com/docs/houdini14.0/vex/functions/fit)
+[fit01](http://www.sidefx.com/docs/houdini14.0/vex/functions/fit01)
+[fit10](http://www.sidefx.com/docs/houdini14.0/vex/functions/fit10)
+[fit11](http://www.sidefx.com/docs/houdini14.0/vex/functions/fit11)
+[floor](http://www.sidefx.com/docs/houdini14.0/vex/functions/floor)
+[flownoise](http://www.sidefx.com/docs/houdini14.0/vex/functions/flownoise)
+[flowpnoise](http://www.sidefx.com/docs/houdini14.0/vex/functions/flowpnoise)
+[foreach](http://www.sidefx.com/docs/houdini14.0/vex/functions/foreach)
+[forpoints](http://www.sidefx.com/docs/houdini14.0/vex/functions/forpoints)
+[frac](http://www.sidefx.com/docs/houdini14.0/vex/functions/frac)
+[fresnel](http://www.sidefx.com/docs/houdini14.0/vex/functions/fresnel)
+[fromNDC](http://www.sidefx.com/docs/houdini14.0/vex/functions/fromNDC)
+[frontface](http://www.sidefx.com/docs/houdini14.0/vex/functions/frontface)
+[gather](http://www.sidefx.com/docs/houdini14.0/vex/functions/gather)
+[geoself](http://www.sidefx.com/docs/houdini14.0/vex/functions/geoself)
+[getattribute](http://www.sidefx.com/docs/houdini14.0/vex/functions/getattribute)
+[getbbox](http://www.sidefx.com/docs/houdini14.0/vex/functions/getbbox)
+[getblurP](http://www.sidefx.com/docs/houdini14.0/vex/functions/getblurP)
+[getbounces](http://www.sidefx.com/docs/houdini14.0/vex/functions/getbounces)
+[getbounds](http://www.sidefx.com/docs/houdini14.0/vex/functions/getbounds)
+[getcomp](http://www.sidefx.com/docs/houdini14.0/vex/functions/getcomp)
+[getcomponents](http://www.sidefx.com/docs/houdini14.0/vex/functions/getcomponents)
+[getfogname](http://www.sidefx.com/docs/houdini14.0/vex/functions/getfogname)
+[getglobalraylevel](http://www.sidefx.com/docs/houdini14.0/vex/functions/getglobalraylevel)
+[getlight](http://www.sidefx.com/docs/houdini14.0/vex/functions/getlight)
+[getlightid](http://www.sidefx.com/docs/houdini14.0/vex/functions/getlightid)
+[getlightname](http://www.sidefx.com/docs/houdini14.0/vex/functions/getlightname)
+[getlights](http://www.sidefx.com/docs/houdini14.0/vex/functions/getlights)
+[getlightscope](http://www.sidefx.com/docs/houdini14.0/vex/functions/getlightscope)
+[getmaterial](http://www.sidefx.com/docs/houdini14.0/vex/functions/getmaterial)
+[getobjectname](http://www.sidefx.com/docs/houdini14.0/vex/functions/getobjectname)
+[getphotonlight](http://www.sidefx.com/docs/houdini14.0/vex/functions/getphotonlight)
+[getpointbbox](http://www.sidefx.com/docs/houdini14.0/vex/functions/getpointbbox)
+[getprimid](http://www.sidefx.com/docs/houdini14.0/vex/functions/getprimid)
+[getptextureid](http://www.sidefx.com/docs/houdini14.0/vex/functions/getptextureid)
+[getraylevel](http://www.sidefx.com/docs/houdini14.0/vex/functions/getraylevel)
+[getrayweight](http://www.sidefx.com/docs/houdini14.0/vex/functions/getrayweight)
+[getscope](http://www.sidefx.com/docs/houdini14.0/vex/functions/getscope)
+[getspace](http://www.sidefx.com/docs/houdini14.0/vex/functions/getspace)
+[gradient](http://www.sidefx.com/docs/houdini14.0/vex/functions/gradient)
+[gradient3d](http://www.sidefx.com/docs/houdini14.0/vex/functions/gradient3d)
+[hair](http://www.sidefx.com/docs/houdini14.0/vex/functions/hair)
+[hasdetailattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/hasdetailattrib)
+[haslight](http://www.sidefx.com/docs/houdini14.0/vex/functions/haslight)
+[hasplane](http://www.sidefx.com/docs/houdini14.0/vex/functions/hasplane)
+[haspointattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/haspointattrib)
+[hasprimattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/hasprimattrib)
+[hasvertexattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/hasvertexattrib)
+[hedge_dstpoint](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_dstpoint)
+[hedge_dstvertex](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_dstvertex)
+[hedge_equivcount](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_equivcount)
+[hedge_isequiv](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_isequiv)
+[hedge_isprimary](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_isprimary)
+[hedge_isvalid](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_isvalid)
+[hedge_next](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_next)
+[hedge_nextequiv](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_nextequiv)
+[hedge_postdstpoint](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_postdstpoint)
+[hedge_postdstvertex](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_postdstvertex)
+[hedge_presrcpoint](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_presrcpoint)
+[hedge_presrcvertex](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_presrcvertex)
+[hedge_prev](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_prev)
+[hedge_prim](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_prim)
+[hedge_primary](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_primary)
+[hedge_srcpoint](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_srcpoint)
+[hedge_srcvertex](http://www.sidefx.com/docs/houdini14.0/vex/functions/hedge_srcvertex)
+[henyeygreenstein](http://www.sidefx.com/docs/houdini14.0/vex/functions/henyeygreenstein)
+[hscript_noise](http://www.sidefx.com/docs/houdini14.0/vex/functions/hscript_noise)
+[hscript_rand](http://www.sidefx.com/docs/houdini14.0/vex/functions/hscript_rand)
+[hscript_snoise](http://www.sidefx.com/docs/houdini14.0/vex/functions/hscript_snoise)
+[hscript_sturb](http://www.sidefx.com/docs/houdini14.0/vex/functions/hscript_sturb)
+[hscript_turb](http://www.sidefx.com/docs/houdini14.0/vex/functions/hscript_turb)
+[hsvtorgb](http://www.sidefx.com/docs/houdini14.0/vex/functions/hsvtorgb)
+[iaspect](http://www.sidefx.com/docs/houdini14.0/vex/functions/iaspect)
+[ichname](http://www.sidefx.com/docs/houdini14.0/vex/functions/ichname)
+[ident](http://www.sidefx.com/docs/houdini14.0/vex/functions/ident)
+[iend](http://www.sidefx.com/docs/houdini14.0/vex/functions/iend)
+[iendtime](http://www.sidefx.com/docs/houdini14.0/vex/functions/iendtime)
+[ihasplane](http://www.sidefx.com/docs/houdini14.0/vex/functions/ihasplane)
+[illuminance](http://www.sidefx.com/docs/houdini14.0/vex/functions/illuminance)
+[import](http://www.sidefx.com/docs/houdini14.0/vex/functions/import)
+[importance_light](http://www.sidefx.com/docs/houdini14.0/vex/functions/importance_light)
+[ingroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/ingroup)
+[inpointgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/inpointgroup)
+[inprimgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/inprimgroup)
+[insert](http://www.sidefx.com/docs/houdini14.0/vex/functions/insert)
+[instance](http://www.sidefx.com/docs/houdini14.0/vex/functions/instance)
+[integrate3d](http://www.sidefx.com/docs/houdini14.0/vex/functions/integrate3d)
+[integrate3dClip](http://www.sidefx.com/docs/houdini14.0/vex/functions/integrate3dClip)
+[interpolate](http://www.sidefx.com/docs/houdini14.0/vex/functions/interpolate)
+[intersect](http://www.sidefx.com/docs/houdini14.0/vex/functions/intersect)
+[intersect3d](http://www.sidefx.com/docs/houdini14.0/vex/functions/intersect3d)
+[intersect_all](http://www.sidefx.com/docs/houdini14.0/vex/functions/intersect_all)
+[intersect_lights](http://www.sidefx.com/docs/houdini14.0/vex/functions/intersect_lights)
+[inumplanes](http://www.sidefx.com/docs/houdini14.0/vex/functions/inumplanes)
+[invert](http://www.sidefx.com/docs/houdini14.0/vex/functions/invert)
+[iplaneindex](http://www.sidefx.com/docs/houdini14.0/vex/functions/iplaneindex)
+[iplanename](http://www.sidefx.com/docs/houdini14.0/vex/functions/iplanename)
+[iplanesize](http://www.sidefx.com/docs/houdini14.0/vex/functions/iplanesize)
+[irate](http://www.sidefx.com/docs/houdini14.0/vex/functions/irate)
+[irradiance](http://www.sidefx.com/docs/houdini14.0/vex/functions/irradiance)
+[isalpha](http://www.sidefx.com/docs/houdini14.0/vex/functions/isalpha)
+[isbound](http://www.sidefx.com/docs/houdini14.0/vex/functions/isbound)
+[isconnected](http://www.sidefx.com/docs/houdini14.0/vex/functions/isconnected)
+[isdigit](http://www.sidefx.com/docs/houdini14.0/vex/functions/isdigit)
+[isfinite](http://www.sidefx.com/docs/houdini14.0/vex/functions/isfinite)
+[isfogray](http://www.sidefx.com/docs/houdini14.0/vex/functions/isfogray)
+[isframes](http://www.sidefx.com/docs/houdini14.0/vex/functions/isframes)
+[isnan](http://www.sidefx.com/docs/houdini14.0/vex/functions/isnan)
+[isotropic](http://www.sidefx.com/docs/houdini14.0/vex/functions/isotropic)
+[israytracing](http://www.sidefx.com/docs/houdini14.0/vex/functions/israytracing)
+[issamples](http://www.sidefx.com/docs/houdini14.0/vex/functions/issamples)
+[isseconds](http://www.sidefx.com/docs/houdini14.0/vex/functions/isseconds)
+[isshadowray](http://www.sidefx.com/docs/houdini14.0/vex/functions/isshadowray)
+[istart](http://www.sidefx.com/docs/houdini14.0/vex/functions/istart)
+[istarttime](http://www.sidefx.com/docs/houdini14.0/vex/functions/istarttime)
+[istracing](http://www.sidefx.com/docs/houdini14.0/vex/functions/istracing)
+[isuvrendering](http://www.sidefx.com/docs/houdini14.0/vex/functions/isuvrendering)
+[isvalidindex](http://www.sidefx.com/docs/houdini14.0/vex/functions/isvalidindex)
+[isvarying](http://www.sidefx.com/docs/houdini14.0/vex/functions/isvarying)
+[itoa](http://www.sidefx.com/docs/houdini14.0/vex/functions/itoa)
+[ixres](http://www.sidefx.com/docs/houdini14.0/vex/functions/ixres)
+[iyres](http://www.sidefx.com/docs/houdini14.0/vex/functions/iyres)
+[join](http://www.sidefx.com/docs/houdini14.0/vex/functions/join)
+[kspline](http://www.sidefx.com/docs/houdini14.0/vex/functions/kspline)
+[len](http://www.sidefx.com/docs/houdini14.0/vex/functions/len)
+[length](http://www.sidefx.com/docs/houdini14.0/vex/functions/length)
+[length2](http://www.sidefx.com/docs/houdini14.0/vex/functions/length2)
+[lerp](http://www.sidefx.com/docs/houdini14.0/vex/functions/lerp)
+[lightid](http://www.sidefx.com/docs/houdini14.0/vex/functions/lightid)
+[limit_sample_space](http://www.sidefx.com/docs/houdini14.0/vex/functions/limit_sample_space)
+[limport](http://www.sidefx.com/docs/houdini14.0/vex/functions/limport)
+[lkspline](http://www.sidefx.com/docs/houdini14.0/vex/functions/lkspline)
+[log](http://www.sidefx.com/docs/houdini14.0/vex/functions/log)
+[log10](http://www.sidefx.com/docs/houdini14.0/vex/functions/log10)
+[lookat](http://www.sidefx.com/docs/houdini14.0/vex/functions/lookat)
+[lspline](http://www.sidefx.com/docs/houdini14.0/vex/functions/lspline)
+[lstrip](http://www.sidefx.com/docs/houdini14.0/vex/functions/lstrip)
+[luminance](http://www.sidefx.com/docs/houdini14.0/vex/functions/luminance)
+[lumname](http://www.sidefx.com/docs/houdini14.0/vex/functions/lumname)
+[makebasis](http://www.sidefx.com/docs/houdini14.0/vex/functions/makebasis)
+[maketransform](http://www.sidefx.com/docs/houdini14.0/vex/functions/maketransform)
+[maskname](http://www.sidefx.com/docs/houdini14.0/vex/functions/maskname)
+[match](http://www.sidefx.com/docs/houdini14.0/vex/functions/match)
+[matchvex_blinn](http://www.sidefx.com/docs/houdini14.0/vex/functions/matchvex_blinn)
+[matchvex_specular](http://www.sidefx.com/docs/houdini14.0/vex/functions/matchvex_specular)
+[mattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/mattrib)
+[max](http://www.sidefx.com/docs/houdini14.0/vex/functions/max)
+[mdensity](http://www.sidefx.com/docs/houdini14.0/vex/functions/mdensity)
+[metaimport](http://www.sidefx.com/docs/houdini14.0/vex/functions/metaimport)
+[metamarch](http://www.sidefx.com/docs/houdini14.0/vex/functions/metamarch)
+[metanext](http://www.sidefx.com/docs/houdini14.0/vex/functions/metanext)
+[metastart](http://www.sidefx.com/docs/houdini14.0/vex/functions/metastart)
+[metaweight](http://www.sidefx.com/docs/houdini14.0/vex/functions/metaweight)
+[min](http://www.sidefx.com/docs/houdini14.0/vex/functions/min)
+[minpos](http://www.sidefx.com/docs/houdini14.0/vex/functions/minpos)
+[mspace](http://www.sidefx.com/docs/houdini14.0/vex/functions/mspace)
+[nbouncetypes](http://www.sidefx.com/docs/houdini14.0/vex/functions/nbouncetypes)
+[nearpoint](http://www.sidefx.com/docs/houdini14.0/vex/functions/nearpoint)
+[nearpoints](http://www.sidefx.com/docs/houdini14.0/vex/functions/nearpoints)
+[neighbour](http://www.sidefx.com/docs/houdini14.0/vex/functions/neighbour)
+[neighbourcount](http://www.sidefx.com/docs/houdini14.0/vex/functions/neighbourcount)
+[neighbours](http://www.sidefx.com/docs/houdini14.0/vex/functions/neighbours)
+[newgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/newgroup)
+[newsampler](http://www.sidefx.com/docs/houdini14.0/vex/functions/newsampler)
+[nextsample](http://www.sidefx.com/docs/houdini14.0/vex/functions/nextsample)
+[ninput](http://www.sidefx.com/docs/houdini14.0/vex/functions/ninput)
+[noise](http://www.sidefx.com/docs/houdini14.0/vex/functions/noise)
+[noised](http://www.sidefx.com/docs/houdini14.0/vex/functions/noised)
+[normal_bsdf](http://www.sidefx.com/docs/houdini14.0/vex/functions/normal_bsdf)
+[normalize](http://www.sidefx.com/docs/houdini14.0/vex/functions/normalize)
+[normalname](http://www.sidefx.com/docs/houdini14.0/vex/functions/normalname)
+[npoints](http://www.sidefx.com/docs/houdini14.0/vex/functions/npoints)
+[npointsgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/npointsgroup)
+[nprimitives](http://www.sidefx.com/docs/houdini14.0/vex/functions/nprimitives)
+[nprimitivesgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/nprimitivesgroup)
+[nrandom](http://www.sidefx.com/docs/houdini14.0/vex/functions/nrandom)
+[ntransform](http://www.sidefx.com/docs/houdini14.0/vex/functions/ntransform)
+[nuniqueval](http://www.sidefx.com/docs/houdini14.0/vex/functions/nuniqueval)
+[occlusion](http://www.sidefx.com/docs/houdini14.0/vex/functions/occlusion)
+[onoise](http://www.sidefx.com/docs/houdini14.0/vex/functions/onoise)
+[opend](http://www.sidefx.com/docs/houdini14.0/vex/functions/opend)
+[opstart](http://www.sidefx.com/docs/houdini14.0/vex/functions/opstart)
+[optransform](http://www.sidefx.com/docs/houdini14.0/vex/functions/optransform)
+[ord](http://www.sidefx.com/docs/houdini14.0/vex/functions/ord)
+[osd_facecount](http://www.sidefx.com/docs/houdini14.0/vex/functions/osd_facecount)
+[osd_firstpatch](http://www.sidefx.com/docs/houdini14.0/vex/functions/osd_firstpatch)
+[osd_limitsurface](http://www.sidefx.com/docs/houdini14.0/vex/functions/osd_limitsurface)
+[osd_limitsurfacevertex](http://www.sidefx.com/docs/houdini14.0/vex/functions/osd_limitsurfacevertex)
+[osd_patchcount](http://www.sidefx.com/docs/houdini14.0/vex/functions/osd_patchcount)
+[osd_patches](http://www.sidefx.com/docs/houdini14.0/vex/functions/osd_patches)
+[outerproduct](http://www.sidefx.com/docs/houdini14.0/vex/functions/outerproduct)
+[ow_nspace](http://www.sidefx.com/docs/houdini14.0/vex/functions/ow_nspace)
+[ow_space](http://www.sidefx.com/docs/houdini14.0/vex/functions/ow_space)
+[ow_vspace](http://www.sidefx.com/docs/houdini14.0/vex/functions/ow_vspace)
+[pack_inttosafefloat](http://www.sidefx.com/docs/houdini14.0/vex/functions/pack_inttosafefloat)
+[pathtrace](http://www.sidefx.com/docs/houdini14.0/vex/functions/pathtrace)
+[pcclose](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcclose)
+[pcconvex](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcconvex)
+[pcexport](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcexport)
+[pcfarthest](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcfarthest)
+[pcfilter](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcfilter)
+[pcfind](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcfind)
+[pcfind_radius](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcfind_radius)
+[pcgenerate](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcgenerate)
+[pcimport](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcimport)
+[pcimportbyidx3](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcimportbyidx3)
+[pcimportbyidx4](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcimportbyidx4)
+[pcimportbyidxf](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcimportbyidxf)
+[pcimportbyidxi](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcimportbyidxi)
+[pcimportbyidxp](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcimportbyidxp)
+[pcimportbyidxs](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcimportbyidxs)
+[pcimportbyidxv](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcimportbyidxv)
+[pciterate](http://www.sidefx.com/docs/houdini14.0/vex/functions/pciterate)
+[pcnumfound](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcnumfound)
+[pcopen](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcopen)
+[pcopenlod](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcopenlod)
+[pcsampleleaf](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcsampleleaf)
+[pcsize](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcsize)
+[pcunshaded](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcunshaded)
+[pcwrite](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcwrite)
+[phong](http://www.sidefx.com/docs/houdini14.0/vex/functions/phong)
+[phongBRDF](http://www.sidefx.com/docs/houdini14.0/vex/functions/phongBRDF)
+[phonglobe](http://www.sidefx.com/docs/houdini14.0/vex/functions/phonglobe)
+[photonmap](http://www.sidefx.com/docs/houdini14.0/vex/functions/photonmap)
+[planeindex](http://www.sidefx.com/docs/houdini14.0/vex/functions/planeindex)
+[planename](http://www.sidefx.com/docs/houdini14.0/vex/functions/planename)
+[planesize](http://www.sidefx.com/docs/houdini14.0/vex/functions/planesize)
+[pluralize](http://www.sidefx.com/docs/houdini14.0/vex/functions/pluralize)
+[pnoise](http://www.sidefx.com/docs/houdini14.0/vex/functions/pnoise)
+[point](http://www.sidefx.com/docs/houdini14.0/vex/functions/point)
+[pointattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointattrib)
+[pointattribsize](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointattribsize)
+[pointattribtype](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointattribtype)
+[pointattribtypeinfo](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointattribtypeinfo)
+[pointedge](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointedge)
+[pointhedge](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointhedge)
+[pointhedgenext](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointhedgenext)
+[pointname](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointname)
+[pointprims](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointprims)
+[pointvertex](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointvertex)
+[pointvertices](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointvertices)
+[polardecomp](http://www.sidefx.com/docs/houdini14.0/vex/functions/polardecomp)
+[pop](http://www.sidefx.com/docs/houdini14.0/vex/functions/pop)
+[pow](http://www.sidefx.com/docs/houdini14.0/vex/functions/pow)
+[prim](http://www.sidefx.com/docs/houdini14.0/vex/functions/prim)
+[prim_attribute](http://www.sidefx.com/docs/houdini14.0/vex/functions/prim_attribute)
+[prim_normal](http://www.sidefx.com/docs/houdini14.0/vex/functions/prim_normal)
+[primattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/primattrib)
+[primattribsize](http://www.sidefx.com/docs/houdini14.0/vex/functions/primattribsize)
+[primattribtype](http://www.sidefx.com/docs/houdini14.0/vex/functions/primattribtype)
+[primattribtypeinfo](http://www.sidefx.com/docs/houdini14.0/vex/functions/primattribtypeinfo)
+[primhedge](http://www.sidefx.com/docs/houdini14.0/vex/functions/primhedge)
+[primintrinsic](http://www.sidefx.com/docs/houdini14.0/vex/functions/primintrinsic)
+[primpoint](http://www.sidefx.com/docs/houdini14.0/vex/functions/primpoint)
+[primpoints](http://www.sidefx.com/docs/houdini14.0/vex/functions/primpoints)
+[primuv](http://www.sidefx.com/docs/houdini14.0/vex/functions/primuv)
+[primvertex](http://www.sidefx.com/docs/houdini14.0/vex/functions/primvertex)
+[primvertexcount](http://www.sidefx.com/docs/houdini14.0/vex/functions/primvertexcount)
+[primvertices](http://www.sidefx.com/docs/houdini14.0/vex/functions/primvertices)
+[print_once](http://www.sidefx.com/docs/houdini14.0/vex/functions/print_once)
+[printf](http://www.sidefx.com/docs/houdini14.0/vex/functions/printf)
+[ptexture](http://www.sidefx.com/docs/houdini14.0/vex/functions/ptexture)
+[ptlined](http://www.sidefx.com/docs/houdini14.0/vex/functions/ptlined)
+[ptransform](http://www.sidefx.com/docs/houdini14.0/vex/functions/ptransform)
+[push](http://www.sidefx.com/docs/houdini14.0/vex/functions/push)
+[qconvert](http://www.sidefx.com/docs/houdini14.0/vex/functions/qconvert)
+[qdistance](http://www.sidefx.com/docs/houdini14.0/vex/functions/qdistance)
+[qinvert](http://www.sidefx.com/docs/houdini14.0/vex/functions/qinvert)
+[qmultiply](http://www.sidefx.com/docs/houdini14.0/vex/functions/qmultiply)
+[qrotate](http://www.sidefx.com/docs/houdini14.0/vex/functions/qrotate)
+[quaternion](http://www.sidefx.com/docs/houdini14.0/vex/functions/quaternion)
+[radians](http://www.sidefx.com/docs/houdini14.0/vex/functions/radians)
+[rand](http://www.sidefx.com/docs/houdini14.0/vex/functions/rand)
+[random](http://www.sidefx.com/docs/houdini14.0/vex/functions/random)
+[random_fhash](http://www.sidefx.com/docs/houdini14.0/vex/functions/random_fhash)
+[random_ihash](http://www.sidefx.com/docs/houdini14.0/vex/functions/random_ihash)
+[random_shash](http://www.sidefx.com/docs/houdini14.0/vex/functions/random_shash)
+[random_sobol](http://www.sidefx.com/docs/houdini14.0/vex/functions/random_sobol)
+[rawbumpmap](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmap)
+[rawbumpmapA](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmapA)
+[rawbumpmapB](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmapB)
+[rawbumpmapG](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmapG)
+[rawbumpmapL](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmapL)
+[rawbumpmapR](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmapR)
+[rawcolormap](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawcolormap)
+[rayhittest](http://www.sidefx.com/docs/houdini14.0/vex/functions/rayhittest)
+[rayimport](http://www.sidefx.com/docs/houdini14.0/vex/functions/rayimport)
+[re_find](http://www.sidefx.com/docs/houdini14.0/vex/functions/re_find)
+[re_findall](http://www.sidefx.com/docs/houdini14.0/vex/functions/re_findall)
+[re_match](http://www.sidefx.com/docs/houdini14.0/vex/functions/re_match)
+[re_replace](http://www.sidefx.com/docs/houdini14.0/vex/functions/re_replace)
+[re_split](http://www.sidefx.com/docs/houdini14.0/vex/functions/re_split)
+[reflect](http://www.sidefx.com/docs/houdini14.0/vex/functions/reflect)
+[reflectlight](http://www.sidefx.com/docs/houdini14.0/vex/functions/reflectlight)
+[refract](http://www.sidefx.com/docs/houdini14.0/vex/functions/refract)
+[refractlight](http://www.sidefx.com/docs/houdini14.0/vex/functions/refractlight)
+[relbbox](http://www.sidefx.com/docs/houdini14.0/vex/functions/relbbox)
+[relpointbbox](http://www.sidefx.com/docs/houdini14.0/vex/functions/relpointbbox)
+[removegroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/removegroup)
+[removeindex](http://www.sidefx.com/docs/houdini14.0/vex/functions/removeindex)
+[removepoint](http://www.sidefx.com/docs/houdini14.0/vex/functions/removepoint)
+[removeprim](http://www.sidefx.com/docs/houdini14.0/vex/functions/removeprim)
+[removevalue](http://www.sidefx.com/docs/houdini14.0/vex/functions/removevalue)
+[renderstate](http://www.sidefx.com/docs/houdini14.0/vex/functions/renderstate)
+[reorder](http://www.sidefx.com/docs/houdini14.0/vex/functions/reorder)
+[resize](http://www.sidefx.com/docs/houdini14.0/vex/functions/resize)
+[resolvemissedray](http://www.sidefx.com/docs/houdini14.0/vex/functions/resolvemissedray)
+[reverse](http://www.sidefx.com/docs/houdini14.0/vex/functions/reverse)
+[rgbtohsv](http://www.sidefx.com/docs/houdini14.0/vex/functions/rgbtohsv)
+[rgbtoxyz](http://www.sidefx.com/docs/houdini14.0/vex/functions/rgbtoxyz)
+[rint](http://www.sidefx.com/docs/houdini14.0/vex/functions/rint)
+[rotate](http://www.sidefx.com/docs/houdini14.0/vex/functions/rotate)
+[rotate_x_to](http://www.sidefx.com/docs/houdini14.0/vex/functions/rotate_x_to)
+[rstrip](http://www.sidefx.com/docs/houdini14.0/vex/functions/rstrip)
+[sample_bsdf](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_bsdf)
+[sample_cauchy](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_cauchy)
+[sample_circle_arc](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_circle_arc)
+[sample_circle_edge_uniform](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_circle_edge_uniform)
+[sample_circle_slice](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_circle_slice)
+[sample_circle_uniform](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_circle_uniform)
+[sample_direction_cone](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_direction_cone)
+[sample_direction_uniform](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_direction_uniform)
+[sample_discrete](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_discrete)
+[sample_exponential](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_exponential)
+[sample_geometry](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_geometry)
+[sample_hemisphere](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_hemisphere)
+[sample_hypersphere_cone](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_hypersphere_cone)
+[sample_hypersphere_uniform](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_hypersphere_uniform)
+[sample_light](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_light)
+[sample_lognormal](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_lognormal)
+[sample_lognormal_by_median](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_lognormal_by_median)
+[sample_normal](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_normal)
+[sample_orientation_cone](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_orientation_cone)
+[sample_orientation_uniform](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_orientation_uniform)
+[sample_photon](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_photon)
+[sample_sphere_cone](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_sphere_cone)
+[sample_sphere_uniform](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_sphere_uniform)
+[sampledisk](http://www.sidefx.com/docs/houdini14.0/vex/functions/sampledisk)
+[scale](http://www.sidefx.com/docs/houdini14.0/vex/functions/scale)
+[select](http://www.sidefx.com/docs/houdini14.0/vex/functions/select)
+[sensor_panorama_create](http://www.sidefx.com/docs/houdini14.0/vex/functions/sensor_panorama_create)
+[sensor_panorama_getcolor](http://www.sidefx.com/docs/houdini14.0/vex/functions/sensor_panorama_getcolor)
+[sensor_panorama_getcone](http://www.sidefx.com/docs/houdini14.0/vex/functions/sensor_panorama_getcone)
+[sensor_panorama_getdepth](http://www.sidefx.com/docs/houdini14.0/vex/functions/sensor_panorama_getdepth)
+[sensor_save](http://www.sidefx.com/docs/houdini14.0/vex/functions/sensor_save)
+[serialize](http://www.sidefx.com/docs/houdini14.0/vex/functions/serialize)
+[set](http://www.sidefx.com/docs/houdini14.0/vex/functions/set)
+[setagentclipnames](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentclipnames)
+[setagentcliptimes](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentcliptimes)
+[setagentclipweights](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentclipweights)
+[setagentcollisionlayer](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentcollisionlayer)
+[setagentcurrentlayer](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentcurrentlayer)
+[setagentlocaltransform](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentlocaltransform)
+[setagentlocaltransforms](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentlocaltransforms)
+[setagentworldtransform](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentworldtransform)
+[setagentworldtransforms](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentworldtransforms)
+[setattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/setattrib)
+[setattribtypeinfo](http://www.sidefx.com/docs/houdini14.0/vex/functions/setattribtypeinfo)
+[setcomp](http://www.sidefx.com/docs/houdini14.0/vex/functions/setcomp)
+[setcurrentlight](http://www.sidefx.com/docs/houdini14.0/vex/functions/setcurrentlight)
+[setdetailattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/setdetailattrib)
+[setpointattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/setpointattrib)
+[setpointgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/setpointgroup)
+[setprimattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/setprimattrib)
+[setprimgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/setprimgroup)
+[setprimintrinsic](http://www.sidefx.com/docs/houdini14.0/vex/functions/setprimintrinsic)
+[setprimvertex](http://www.sidefx.com/docs/houdini14.0/vex/functions/setprimvertex)
+[setvertexattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/setvertexattrib)
+[shadow](http://www.sidefx.com/docs/houdini14.0/vex/functions/shadow)
+[shadow_light](http://www.sidefx.com/docs/houdini14.0/vex/functions/shadow_light)
+[shadowmap](http://www.sidefx.com/docs/houdini14.0/vex/functions/shadowmap)
+[shimport](http://www.sidefx.com/docs/houdini14.0/vex/functions/shimport)
+[shl](http://www.sidefx.com/docs/houdini14.0/vex/functions/shl)
+[shr](http://www.sidefx.com/docs/houdini14.0/vex/functions/shr)
+[shrz](http://www.sidefx.com/docs/houdini14.0/vex/functions/shrz)
+[sign](http://www.sidefx.com/docs/houdini14.0/vex/functions/sign)
+[simport](http://www.sidefx.com/docs/houdini14.0/vex/functions/simport)
+[sin](http://www.sidefx.com/docs/houdini14.0/vex/functions/sin)
+[sinh](http://www.sidefx.com/docs/houdini14.0/vex/functions/sinh)
+[sleep](http://www.sidefx.com/docs/houdini14.0/vex/functions/sleep)
+[slerp](http://www.sidefx.com/docs/houdini14.0/vex/functions/slerp)
+[slice](http://www.sidefx.com/docs/houdini14.0/vex/functions/slice)
+[smooth](http://www.sidefx.com/docs/houdini14.0/vex/functions/smooth)
+[snoise](http://www.sidefx.com/docs/houdini14.0/vex/functions/snoise)
+[solvecubic](http://www.sidefx.com/docs/houdini14.0/vex/functions/solvecubic)
+[solvepoly](http://www.sidefx.com/docs/houdini14.0/vex/functions/solvepoly)
+[solvequadratic](http://www.sidefx.com/docs/houdini14.0/vex/functions/solvequadratic)
+[sort](http://www.sidefx.com/docs/houdini14.0/vex/functions/sort)
+[specular](http://www.sidefx.com/docs/houdini14.0/vex/functions/specular)
+[specularBRDF](http://www.sidefx.com/docs/houdini14.0/vex/functions/specularBRDF)
+[spline](http://www.sidefx.com/docs/houdini14.0/vex/functions/spline)
+[split](http://www.sidefx.com/docs/houdini14.0/vex/functions/split)
+[sprintf](http://www.sidefx.com/docs/houdini14.0/vex/functions/sprintf)
+[sqrt](http://www.sidefx.com/docs/houdini14.0/vex/functions/sqrt)
+[startswith](http://www.sidefx.com/docs/houdini14.0/vex/functions/startswith)
+[storelightexport](http://www.sidefx.com/docs/houdini14.0/vex/functions/storelightexport)
+[strip](http://www.sidefx.com/docs/houdini14.0/vex/functions/strip)
+[strlen](http://www.sidefx.com/docs/houdini14.0/vex/functions/strlen)
+[switch](http://www.sidefx.com/docs/houdini14.0/vex/functions/switch)
+[swizzle](http://www.sidefx.com/docs/houdini14.0/vex/functions/swizzle)
+[tan](http://www.sidefx.com/docs/houdini14.0/vex/functions/tan)
+[tanh](http://www.sidefx.com/docs/houdini14.0/vex/functions/tanh)
+[tet_adjacent](http://www.sidefx.com/docs/houdini14.0/vex/functions/tet_adjacent)
+[tet_faceindex](http://www.sidefx.com/docs/houdini14.0/vex/functions/tet_faceindex)
+[teximport](http://www.sidefx.com/docs/houdini14.0/vex/functions/teximport)
+[texprintf](http://www.sidefx.com/docs/houdini14.0/vex/functions/texprintf)
+[texture](http://www.sidefx.com/docs/houdini14.0/vex/functions/texture)
+[texture3d](http://www.sidefx.com/docs/houdini14.0/vex/functions/texture3d)
+[texture3dBox](http://www.sidefx.com/docs/houdini14.0/vex/functions/texture3dBox)
+[titlecase](http://www.sidefx.com/docs/houdini14.0/vex/functions/titlecase)
+[toNDC](http://www.sidefx.com/docs/houdini14.0/vex/functions/toNDC)
+[tolower](http://www.sidefx.com/docs/houdini14.0/vex/functions/tolower)
+[toupper](http://www.sidefx.com/docs/houdini14.0/vex/functions/toupper)
+[trace](http://www.sidefx.com/docs/houdini14.0/vex/functions/trace)
+[translate](http://www.sidefx.com/docs/houdini14.0/vex/functions/translate)
+[translucent](http://www.sidefx.com/docs/houdini14.0/vex/functions/translucent)
+[transpose](http://www.sidefx.com/docs/houdini14.0/vex/functions/transpose)
+[trunc](http://www.sidefx.com/docs/houdini14.0/vex/functions/trunc)
+[tw_nspace](http://www.sidefx.com/docs/houdini14.0/vex/functions/tw_nspace)
+[tw_space](http://www.sidefx.com/docs/houdini14.0/vex/functions/tw_space)
+[tw_vspace](http://www.sidefx.com/docs/houdini14.0/vex/functions/tw_vspace)
+[uniqueval](http://www.sidefx.com/docs/houdini14.0/vex/functions/uniqueval)
+[unpack_intfromsafefloat](http://www.sidefx.com/docs/houdini14.0/vex/functions/unpack_intfromsafefloat)
+[unserialize](http://www.sidefx.com/docs/houdini14.0/vex/functions/unserialize)
+[upush](http://www.sidefx.com/docs/houdini14.0/vex/functions/upush)
+[variadicarguments](http://www.sidefx.com/docs/houdini14.0/vex/functions/variadicarguments)
+[variance](http://www.sidefx.com/docs/houdini14.0/vex/functions/variance)
+[velocityname](http://www.sidefx.com/docs/houdini14.0/vex/functions/velocityname)
+[vertex](http://www.sidefx.com/docs/houdini14.0/vex/functions/vertex)
+[vertexattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/vertexattrib)
+[vertexattribsize](http://www.sidefx.com/docs/houdini14.0/vex/functions/vertexattribsize)
+[vertexattribtype](http://www.sidefx.com/docs/houdini14.0/vex/functions/vertexattribtype)
+[vertexattribtypeinfo](http://www.sidefx.com/docs/houdini14.0/vex/functions/vertexattribtypeinfo)
+[vertexhedge](http://www.sidefx.com/docs/houdini14.0/vex/functions/vertexhedge)
+[vertexindex](http://www.sidefx.com/docs/houdini14.0/vex/functions/vertexindex)
+[vertexnext](http://www.sidefx.com/docs/houdini14.0/vex/functions/vertexnext)
+[vertexpoint](http://www.sidefx.com/docs/houdini14.0/vex/functions/vertexpoint)
+[vertexprev](http://www.sidefx.com/docs/houdini14.0/vex/functions/vertexprev)
+[vertexprim](http://www.sidefx.com/docs/houdini14.0/vex/functions/vertexprim)
+[vertexprimindex](http://www.sidefx.com/docs/houdini14.0/vex/functions/vertexprimindex)
+[vnoise](http://www.sidefx.com/docs/houdini14.0/vex/functions/vnoise)
+[volume](http://www.sidefx.com/docs/houdini14.0/vex/functions/volume)
+[volumegradient](http://www.sidefx.com/docs/houdini14.0/vex/functions/volumegradient)
+[volumeindex](http://www.sidefx.com/docs/houdini14.0/vex/functions/volumeindex)
+[volumeindexorigin](http://www.sidefx.com/docs/houdini14.0/vex/functions/volumeindexorigin)
+[volumeindextopos](http://www.sidefx.com/docs/houdini14.0/vex/functions/volumeindextopos)
+[volumeindexv](http://www.sidefx.com/docs/houdini14.0/vex/functions/volumeindexv)
+[volumepostoindex](http://www.sidefx.com/docs/houdini14.0/vex/functions/volumepostoindex)
+[volumeres](http://www.sidefx.com/docs/houdini14.0/vex/functions/volumeres)
+[volumesample](http://www.sidefx.com/docs/houdini14.0/vex/functions/volumesample)
+[volumesamplev](http://www.sidefx.com/docs/houdini14.0/vex/functions/volumesamplev)
+[vtransform](http://www.sidefx.com/docs/houdini14.0/vex/functions/vtransform)
+[wireblinn](http://www.sidefx.com/docs/houdini14.0/vex/functions/wireblinn)
+[wirediffuse](http://www.sidefx.com/docs/houdini14.0/vex/functions/wirediffuse)
+[wnoise](http://www.sidefx.com/docs/houdini14.0/vex/functions/wnoise)
+[wo_nspace](http://www.sidefx.com/docs/houdini14.0/vex/functions/wo_nspace)
+[wo_space](http://www.sidefx.com/docs/houdini14.0/vex/functions/wo_space)
+[wo_vspace](http://www.sidefx.com/docs/houdini14.0/vex/functions/wo_vspace)
+[writepixel](http://www.sidefx.com/docs/houdini14.0/vex/functions/writepixel)
+[wt_nspace](http://www.sidefx.com/docs/houdini14.0/vex/functions/wt_nspace)
+[wt_space](http://www.sidefx.com/docs/houdini14.0/vex/functions/wt_space)
+[wt_vspace](http://www.sidefx.com/docs/houdini14.0/vex/functions/wt_vspace)
+[xnoise](http://www.sidefx.com/docs/houdini14.0/vex/functions/xnoise)
+[xnoised](http://www.sidefx.com/docs/houdini14.0/vex/functions/xnoised)
+[xyzdist](http://www.sidefx.com/docs/houdini14.0/vex/functions/xyzdist)
+[xyztorgb](http://www.sidefx.com/docs/houdini14.0/vex/functions/xyztorgb)

Total: 637 functions.
    
#### New in 13.0



Total: 0 functions.
    
#### New in 12.5



Total: 0 functions.
    
#### New in 12.1



Total: 0 functions.
    
#### New in 12.0



Total: 0 functions.
    
#### New in 11.0



Total: 0 functions.
    
#### From 10.0



Total: 0 functions.